![alt text](./pageheader_rose2_babies.jpg)

# ANALYZING VENTILATOR DATA DG004

**Author: Dr Gusztav Belteki**

This notebook contains the Python code to produce the data and the illustrations 
published in the paper:

Panayiotou E, Spike K, Morley C, Belteki G. **Ventilator respiratory graphic
diagnosis of hiccupping in non-ketotic hyperglycinaemia.** _BMJ Case Reports_ 
2017 Aug9;2017. pii: bcr-2017-220267.

Contact: gbelteki@aol.com

### Importing the necessary libraries and setting options

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from pprint import pprint
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

### Clinical details

In [ ]:
clin_details = pd.read_excel('/Users/guszti/Desktop/ventilation_data/data_grabber_patient_data.xlsx', 
                             sheetname = 'DG004')

In [ ]:
clin_details

In [ ]:
gestation = clin_details.ix[0, 1]; bw = clin_details.ix[2, 1]/1000; cw = clin_details.ix[4, 1]/1000
gestation

In [ ]:
bw

In [ ]:
cw

In [ ]:
dg = clin_details.ix[5, 1].split(',')
dg

In [ ]:
comments = clin_details.ix[9, 1].split(',')
comments

### Blood gases

In [ ]:
gases = pd.read_excel('/Users/guszti/Desktop/ventilation_data/data_grabber_gases.xlsx', sheetname = 'DG004')
gases = gases.T
gases.columns = gases.ix[0]
gases = gases.reset_index()

gases[[0,1,5,6,7,8,9,14,30]]



### Importing and cleaning up the ventilator data

### List and set the working directory

In [ ]:
%cd /Users/guszti/Desktop/ventilation_data/DG004
cwd = '/Users/guszti/Desktop/ventilation_data/DG004'
!ls -l

In [ ]:
fnames = ['CsvLogBase_2015-10-12_085111.498_' , 'CsvLogBase_2015-10-12_175541.015_']

In [ ]:
dir_write = '/Users/guszti/Desktop/ventilation_data/Analyses/DG004'

In [ ]:
fnames2 = 'DG004_'

###  Importing fast data

In [ ]:
fast_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'fast_Unknown.csv'), parse_dates = [[1, 2]])
fast_1.columns = ['date_time','time_ms', 'rel_time_s', 'paw', 'flow', 'volume']
fast_1.index = fast_1.date_time
fast_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'fast_Unknown.csv'), parse_dates = [[1, 2]])
fast_2.columns = ['date_time','time_ms', 'rel_time_s', 'paw', 'flow', 'volume']
fast_2.index = fast_2.date_time

In [ ]:
fast_1.head()

In [ ]:
fast_1.tail()

In [ ]:
fast_2.head()

In [ ]:
fast_2.tail()

In [ ]:
fast = pd.concat([fast_1, fast_2])
del fast_1
del fast_2

In [ ]:
fast.info()

In [ ]:
fast.head()

In [ ]:
fast.tail()

### Importing ventilator settings

In [ ]:
slow_Setting_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'slow_Setting.csv'), parse_dates = [[1, 2]])
slow_Setting_1.index = slow_Setting_1.Date_Time
slow_Setting_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'slow_Setting.csv'), parse_dates = [[1, 2]])
slow_Setting_2.index = slow_Setting_2.Date_Time

In [ ]:
slow_Setting = pd.concat([slow_Setting_1, slow_Setting_2])
del slow_Setting_1
del slow_Setting_2

In [ ]:
slow_Setting;

In [ ]:
a = slow_Setting[slow_Setting.Id != 'FiO2']
a = a[a.Id != 'Ø tube']
a = a[a.Id != 'Tapn']
a = a[a.Id != 'ΔPsupp']
a = a[a.Id != 'Tube Æ']
a = a[a.Id != 'RRsigh']
a = a[a.Id != 'Psigh']
a = a[a.Id != 'Slopesigh']
a = a[a.Unit != 'L']
a = a[a.Id != 'I (I:E)']
a = a[a.Id != 'E (I:E)']
a = a[a.Id != 'Te']
a = a[a.Id != 'Ti']
a = a[a.Id != 'MVlow delay']
a = a[a.Id != 'MVhigh delay']
a.drop_duplicates(["Rel.Time [s]", "Name"], inplace = True)
slow_Setting_selected = a

In [ ]:
slow_Setting_selected

In [ ]:
slow_Text_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'slow_Text.csv'), parse_dates = [[1, 2]])
slow_Text_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'slow_Text.csv'), parse_dates = [[1, 2]])
slow_Text_1.index = slow_Text_1.Date_Time
slow_Text_2.index = slow_Text_2.Date_Time

In [ ]:
slow_Text = pd.concat([slow_Text_1, slow_Text_2])
del slow_Text_1
del slow_Text_2

In [ ]:
a = slow_Text[slow_Text.Id != 'Device is in neonatal mode']
a = a[a.Id != 'Device is in neonatal mode']
a = a[a.Id != 'Selected CO2 unit is mmHg']
a = a[a.Id != "Selected language (variable text string transmitted with this code, see 'MEDIBUS.X, Rules and Standards for Implementation' for further information)"]
a = a[a.Id != 'Device configured for intubated patient ventilation']
a = a[a.Id != 'Active humid heated']
a = a[a.Id != 'Audio pause active']
a = a[a.Id != 'Active humid unheated']
a = a[a.Id != 'Suction maneuver active']
a = a[a.Id != 'Tube type endotracheal']
a = a[a.Id != 'Tracheal pressure calculation enabled as real-time value (independent of ATC adjunct)']
a.drop_duplicates(["Rel.Time [s]", "Id"], inplace = True)
slow_Text_selected = a

In [ ]:
slow_Text.index = range(len(slow_Text))
slow_Text;

In [ ]:
slow_Text_selected

### Calculate recording period

In [ ]:
recording_period = (fast.index[-1] - fast.index[0]).total_seconds()
recording_period

In [ ]:
recording_period_2 = (fast.index[-1] - fast.index[0])
print(recording_period_2)

### Calculate descriptive statistics about, PAW, FLOW, VOLUME

In [ ]:
def par_stats(par):
    '''
    input: DataFrame containing 1 column of data 
    This function return detailed statistics about the reduced data ;
    - number of data points in the aggregated data set, mean, standard deviation, median, mean absolute deviation,
      minium,5th centila, 25th centile, 75th centile, 95th centile, maximum of the time periods over the recording time 
    '''
    return (par.size, par.mean(), par.std(), par.median(), par.mad(), par.min(), par.quantile(0.05), 
            par.quantile(0.25), par.quantile(0.75), par.quantile(0.95), par.max())

In [ ]:
fast['flow_abs'] = abs(fast['flow'])

In [ ]:
paw_stats = par_stats(fast['paw'])
flow_stats = par_stats(fast['flow_abs']) 
volume_stats = par_stats(fast['volume'])

In [ ]:
paw_stats = DataFrame([paw_stats], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
flow_stats = DataFrame([flow_stats], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
volume_stats = DataFrame([volume_stats], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])

In [ ]:
paw_stats

In [ ]:
flow_stats

In [ ]:
volume_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s%s' % (dir_write, fnames2, 'fast_data_stats.xlsx'))
paw_stats.to_excel(writer, 'paw')
flow_stats.to_excel(writer, 'flow')
volume_stats.to_excel(writer, 'volume')
writer.save()

### Check how many duplicated time stamps

In [ ]:
mask = fast.duplicated(subset = 'date_time')

In [ ]:
len(mask)

In [ ]:
uniq = fast[~mask]

In [ ]:
len(uniq)

In [ ]:
dupl = fast[mask]

In [ ]:
len(dupl)

### Reindex the duplicates 

#### using the timestamp-shifting method. This method is shifting the timestamp of the duplicates up by 8 msec

In [ ]:
dupl.index = dupl.ix[:].date_time.apply(lambda x: x + pd.offsets.Milli(8))

In [ ]:
dupl.head(10)

In [ ]:
fast_non_dupl = pd.concat([uniq, dupl])

In [ ]:
len(fast_non_dupl)

In [ ]:
fast_non_dupl = fast_non_dupl.sort()

In [ ]:
fast_non_dupl = fast_non_dupl.drop('date_time', axis = 1)

In [ ]:
fast_non_dupl.head()

In [ ]:
fast = fast_non_dupl

###  Writing data structures into .csv text files - not run

###  Importing the edited .csv text files - not run

### Identifying  and visualizing individual waveforms

In [ ]:
start_1 = '2015-10-12 10:37:00'
end_1 = '2015-10-12 10:37:30'

In [ ]:
start_2 = '2015-10-12 10:30:00'
end_2 = '2015-10-12 10:30:30'

In [ ]:
start_3 = '2015-10-12 10:37:00'
end_3 = '2015-10-12 10:37:10'

In [ ]:
fast_short_1 = fast[start_1 : end_1]

In [ ]:
fast_short_2 = fast[start_2 : end_2]

In [ ]:
fast_short_3 = fast[start_3 : end_3]

In [ ]:
len(fast_short_1), len(fast_short_2), len(fast_short_3)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_1.paw.plot(ax = axes[0], color = 'red', title = 'Pressure',  
                      ylim = [-10, (fast_short_1.paw.max() * 1.2)], fontsize = 12);
fast_short_1.flow.plot(ax = axes[1], color = 'green', title = 'Flow', 
                ylim = [(fast_short_1.flow.min() * 1.2), (fast_short_1.flow.max() * 1.2)], fontsize = 12);
fast_short_1.volume.plot(ax = axes[2], 
                 color = 'blue', title = 'Volume', ylim = [0, (fast_short_1.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 16, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 16, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2a.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_1.paw.plot(ax = axes[0], color = 'black', title = 'Pressure',  
                      ylim = [-10, (fast_short_1.paw.max() * 1.2)], fontsize = 12);
fast_short_1.flow.plot(ax = axes[1], color = 'black', title = 'Flow', 
                ylim = [(fast_short_1.flow.min() * 1.2), (fast_short_1.flow.max() * 1.2)], fontsize = 12);
fast_short_1.volume.plot(ax = axes[2], 
                 color = 'black', title = 'Volume', ylim = [0, (fast_short_1.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 16, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 16, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2a_bw.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(30,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_1.paw.plot(ax = axes[0], color = 'red', title = 'Pressure',  
                      ylim = [-10, (fast_short_1.paw.max() * 1.2)], fontsize = 12);
fast_short_1.flow.plot(ax = axes[1], color = 'green', title = 'Flow', 
                ylim = [(fast_short_1.flow.min() * 1.2), (fast_short_1.flow.max() * 1.2)], fontsize = 12);
fast_short_1.volume.plot(ax = axes[2], 
                 color = 'blue', title = 'Volume', ylim = [0, (fast_short_1.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 16, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 16, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2a_wide.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_2.paw.plot(ax = axes[0], color = 'red', title = 'Pressure',  
                      ylim = [0, (fast_short_2.paw.max() * 1.2)], fontsize = 12);
fast_short_2.flow.plot(ax = axes[1], color = 'green', title = 'Flow', 
                ylim = [(fast_short_2.flow.min() * 1.2), (fast_short_2.flow.max() * 1.2)], fontsize = 12);
fast_short_2.volume.plot(ax = axes[2], color = 'blue', title = 'Volume', 
                         ylim = [0, (fast_short_2.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 14, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'DG004_Figure_2b.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(12,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_2.paw.plot(ax = axes[0], color = 'black', title = 'Pressure',  
                      ylim = [0, (fast_short_2.paw.max() * 1.2)], fontsize = 12);
fast_short_2.flow.plot(ax = axes[1], color = 'black', title = 'Flow', 
                ylim = [(fast_short_2.flow.min() * 1.2), (fast_short_2.flow.max() * 1.2)], fontsize = 12);
fast_short_2.volume.plot(ax = axes[2], color = 'black', title = 'Volume', 
                         ylim = [0, (fast_short_2.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 14, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2b_bw.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(30,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_2.paw.plot(ax = axes[0], color = 'red', title = 'Pressure',  
                      ylim = [0, (fast_short_2.paw.max() * 1.2)], fontsize = 12);
fast_short_2.flow.plot(ax = axes[1], color = 'green', title = 'Flow', 
                ylim = [(fast_short_2.flow.min() * 1.2), (fast_short_2.flow.max() * 1.2)], fontsize = 12);
fast_short_2.volume.plot(ax = axes[2], color = 'blue', title = 'Volume', 
                         ylim = [0, (fast_short_2.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 14, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2b_wide.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex = 'all', sharey = 'none')
fig.set_size_inches(30,9); fig.set_label('res')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
fast_short_3.paw.plot(ax = axes[0], color = 'red', title = 'Pressure',  
                      ylim = [-10, (fast_short_3.paw.max() * 1.2)], fontsize = 12);
fast_short_3.flow.plot(ax = axes[1], color = 'green', title = 'Flow', 
                ylim = [(fast_short_3.flow.min() * 1.2), (fast_short_3.flow.max() * 1.2)], fontsize = 12);
fast_short_3.volume.plot(ax = axes[2], color = 'blue', title = 'Volume', 
                         ylim = [0, (fast_short_3.volume.max() * 1.2)], fontsize = 12, rot = 0);
axes[0].set_xlabel(''); axes[1].set_xlabel(''); axes[2].set_xlabel('Time', size = 16, color = 'black', rotation = 0 )
axes[0].set_ylabel('mbar', size = 14, color = 'black'); axes[1].set_ylabel('L/min', size = 14, color = 'black')
axes[2].set_ylabel('mL', size = 14, color = 'black')

axes[0].set_title('Pressure', size = 16, color = 'black'); axes[1].set_title('Flow', size = 16, color = 'black')
axes[2].set_title('Volume', size = 16, color = 'black')

axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray'); 
axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_2a_wider.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

###  Longer (30 seconds/line) visualization of waveforms

In [ ]:
timelist = (pd.date_range('2015-10-12 10:37:00', '2015-10-12 10:42:00', freq = '10S'))
timelist

In [ ]:
fig = plt.figure()
fig.set_size_inches(15,250)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.2)
j = 1
for i in timelist[:-1]:
    ax1 = fig.add_subplot(len(timelist)*3, 1, j)
    fast.ix[i : i + pd.to_timedelta('00:00:30')].paw.plot(ax = ax1, color = 'red', title = 'Paw', grid = 'on', rot = 0)
    ax1.set_xlabel('')
    ax2 = fig.add_subplot(len(timelist)*3, 1, j+1)
    fast.ix[i : i + pd.to_timedelta('00:00:30')].flow.plot(ax = ax2, color = 'green', title = 'flow', grid = 'on', rot = 0)
    ax2.set_xlabel('')
    ax3 = fig.add_subplot(len(timelist)*3, 1, j+2)
    fast.ix[i : i + pd.to_timedelta('00:00:30')].volume.plot(ax = ax3, color = 'blue', title = 'volume', grid = 'on', rot = 0)
    ax3.set_xlabel('Time')
    j += 3